# Ways to Create object from  FHIR Instance

In [7]:
import sys, os
from pprint import pprint
from json import dumps, loads

### define utility to convert json string to python object using object_hook method - able to access elements using dot notation

In [8]:
def _json_object_hook(d):
     return type('X', (object,), d)()


def json2obj(data):
     return loads(data, object_hook=_json_object_hook)

### Get FHIR Instance as JSON

In [9]:
in_path = '/Users/ehaas/Documents/FHIR/synthea/output/fhir/'
f_name = 'Denisse335_Stracke611_e760e08d-a0ff-4cd7-83c0-36b27dae6ec3'

def open_file(): # get files
    with open(f'{in_path}{f_name}.json',encoding='utf-8', errors='ignore') as f:
        r = f.read()
       
        return(r)
b = open_file() 
#print(b)

### Convert json string to py object

In [10]:
f = json2obj(b)
f

In [11]:
f.resourceType

'Bundle'

In [12]:
f.entry[0].fullUrl = 'foo'
f.entry[0]._fullUrl= 'foo_2'
for i in f.entry:
    print(i.resource.resourceType,i.resource.id)   

Patient a34986ff-de7f-426a-84f5-dd885e1cae09
Organization 49318f80-bd8b-3fc7-a096-ac43088b0c12
Practitioner 163c2352-4a03-4294-a2ad-e987b72596c0
Encounter e7874274-8ae3-4bce-9a37-b907c5797146
Observation 10fc1903-6f1f-4907-9b1d-4f3f278813d4
Observation 4b5f64d4-4cc2-47fd-be18-b6a30fae4a3f
Observation ddb56aea-060d-4475-a83c-bba87e9651dc
Observation 139671c0-a886-4d42-9db1-8437a97b951d
Observation e06771ed-59be-428e-a96a-6302a6b51e5e
Observation a37c7f7a-fd73-491d-82a1-b6ce2451e763
Procedure b3e84463-e638-47fc-9ae7-63a6ff53e51e
Immunization 20370327-8cf7-48c2-beb0-855cf8b54da7
Immunization 1722c183-d8d6-4439-a505-46c8c25f8d54
Claim 8358a694-8f3e-4704-8349-052d8030005c
ExplanationOfBenefit 4a3e4673-cab6-4f5d-833c-c771bfb4787b
Encounter c87e45cb-9078-45bd-a74b-d48b578265ca
Condition 023e0fdc-8ef9-4ed0-b214-8bea51fa706b
Claim db53d98f-f31c-4abb-8241-968e1b26cc6b
ExplanationOfBenefit 9896412d-2a7d-44b8-aa81-f1e9fc6f8a99
Encounter c5fb1a74-e00f-4cc8-ab61-84ab8409a87a
Observation 00383303-8bc

## Try same thing using namedtuples

In [14]:
import namedtupled
b = b.replace("class", "FHIRClass")  # ======= keywords cause problems will need to map back and forth ==========

f = namedtupled.json(b)

for i in f.entry:
    pass
    print(i.resource.resourceType,i.resource.id) 
    # print(dumps(i.resource._asdict(),indent=4))


Patient a34986ff-de7f-426a-84f5-dd885e1cae09
Organization 49318f80-bd8b-3fc7-a096-ac43088b0c12
Practitioner 163c2352-4a03-4294-a2ad-e987b72596c0
Encounter e7874274-8ae3-4bce-9a37-b907c5797146
Observation 10fc1903-6f1f-4907-9b1d-4f3f278813d4
Observation 4b5f64d4-4cc2-47fd-be18-b6a30fae4a3f
Observation ddb56aea-060d-4475-a83c-bba87e9651dc
Observation 139671c0-a886-4d42-9db1-8437a97b951d
Observation e06771ed-59be-428e-a96a-6302a6b51e5e
Observation a37c7f7a-fd73-491d-82a1-b6ce2451e763
Procedure b3e84463-e638-47fc-9ae7-63a6ff53e51e
Immunization 20370327-8cf7-48c2-beb0-855cf8b54da7
Immunization 1722c183-d8d6-4439-a505-46c8c25f8d54
Claim 8358a694-8f3e-4704-8349-052d8030005c
ExplanationOfBenefit 4a3e4673-cab6-4f5d-833c-c771bfb4787b
Encounter c87e45cb-9078-45bd-a74b-d48b578265ca
Condition 023e0fdc-8ef9-4ed0-b214-8bea51fa706b
Claim db53d98f-f31c-4abb-8241-968e1b26cc6b
ExplanationOfBenefit 9896412d-2a7d-44b8-aa81-f1e9fc6f8a99
Encounter c5fb1a74-e00f-4cc8-ab61-84ab8409a87a
Observation 00383303-8bc

### finally use FHIRclient models and dynamically create

In [15]:
# Standard import
import importlib
D = loads(b)
f_type = D['resourceType']

# Instantiate the class (pass arguments to the constructor, if needed)
F = getattr(importlib.import_module(f'fhirclient.r4models.{f_type.lower()}'), f_type)

"""f = F(D, strict = False)
for i in f.entry: # THIS DOES NOT WORK FOR BUNDLE
    print(i.resource.resourceType,i.resource.id) 
    # print(dumps(i.resource._asdict(),indent=4))

print(dumps(bundle.as_json(), indent = 4))  # THIS DOES NOT WORK FOR BUNDLE"""
F

fhirclient.r4models.bundle.Bundle

In [ ]:
def write_file(r_json,type,id): # write file
    out_path = '/Users/ehaas/Documents/FHIR/USCoreR4Validator/source/examples/'  # append forward slash
    with open(f'{out_path}{type}-{id}.json', 'w') as f:
        f.write(r_json)